<a href="https://colab.research.google.com/github/kingbughk/Datathon-2021/blob/main/AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup the connections


In [ ]:
#sets the project id
PROJECT_ID = "" #@param {type:"string"}

In [ ]:
import os
from google.colab import auth

#sets dateset
DATASET_PROJECT_ID = 'amsterdamumcdb-data'
DATASET_ID = 'ams102'
LOCATION = 'eu'

#all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

auth.authenticate_user()
print('Authenticated')

Authenticated


##Import the query tools

In [ ]:
from google.cloud.bigquery import magics
from google.cloud import bigquery

#sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID)
magics.context.default_query_job_config = def_config

#sets client options job configuration
client_options = {}
client_options['location'] = LOCATION
magics.context.bigquery_client_options = client_options

#get the amsterdamumcdb package from PiPy repository for use in Colab
!pip install amsterdamumcdb
import amsterdamumcdb as adb

%load_ext google.colab.data_table

     |████████████████████████████████| 204kB 5.0MB/s 


##Get data involving coded with AF

In [ ]:
%%bigquery af_admissions
    SELECT
      admissionid
      ,MIN(measuredat) AS first_af_time
    FROM listitems
    WHERE 
      listitems.itemid = 6671	--Hartritme
      AND listitems.valueid = 13	--Atr fib
    GROUP BY
      admissionid


#Number of admissions with AF anytime

In [ ]:
af_admissions.count()

admissionid      4086
first_af_time    4086
dtype: int64

In [ ]:
af_admissions.head(10)


,admissionid,first_af_time
0,93,208733520000
1,160,-1500000
2,254,311280000
3,272,23940000
4,295,141120000
5,299,92940000
6,337,53974200000
7,375,425940000
8,423,1937280000
9,481,20400000


#Examine the record of rhythm of admissionid=295 and test linking 2 data tables

In [ ]:
%%bigquery rhythm_of_295
    SELECT
      listitems.admissionid
      ,listitems.measuredat/1000/60/60 AS time_of_record -- converted to hours
      ,listitems.valueid 
      ,admissions.gender
    FROM listitems, admissions
    WHERE 
      itemid = 6671	--Hartritme
      AND listitems.admissionid = 295
      AND listitems.admissionid=admissions.admissionid
    ORDER BY time_of_record

In [ ]:
rhythm_of_295

,admissionid,time_of_record,valueid,gender
0,295,3.033333,20,Vrouw
1,295,4.033333,20,Vrouw
2,295,5.033333,20,Vrouw
3,295,6.033333,20,Vrouw
4,295,7.033333,1,Vrouw
...,...,...,...,...
81,295,91.033333,13,Vrouw
82,295,92.033333,13,Vrouw
83,295,93.033333,13,Vrouw
84,295,94.033333,13,Vrouw


##Examine the types of possible rhythms

In [ ]:
%%bigquery type_of_rhythms
    SELECT DISTINCT
      listitems.valueid
      , listitems.value
    FROM listitems
    WHERE 
      itemid = 6671	--Hartritme

In [ ]:
type_of_rhythms

,valueid,value
0,33,P.M-ritme
1,20,Sinus bra
2,14,Atr flut
3,2,AVblock 1
4,17,Sinus ari
5,22,SVT
6,4,AVblock 3
7,25,VT
8,15,Junct
9,18,Sinus arr


## Examine the possible rhythms from dictionary

In [ ]:
dictionary = adb.get_dictionary()
dictionary[(dictionary["itemid"]==6671)]

,itemid,item,item_en,abbreviation,categoryid,category,category_en,ordercategoryid,ordercategory,islabresult,valueid,value,unitid,unit,low_normal_value,high_normal_value,table,count,count_validated
2359,6671,Hartritme,heart rhythm,Ritme,320,HD-Hartritme,hemodynamics-heart rhythm,NaN,NaN,0.0,1.0,NSR,NaN,NaN,NaN,NaN,listitems,1203779,1203779.0
2360,6671,Hartritme,heart rhythm,Ritme,320,HD-Hartritme,hemodynamics-heart rhythm,NaN,NaN,0.0,2.0,AVblock 1,NaN,NaN,NaN,NaN,listitems,20170,20170.0
2361,6671,Hartritme,heart rhythm,Ritme,320,HD-Hartritme,hemodynamics-heart rhythm,NaN,NaN,0.0,3.0,AVblock 2,NaN,NaN,NaN,NaN,listitems,1305,1305.0
2362,6671,Hartritme,heart rhythm,Ritme,320,HD-Hartritme,hemodynamics-heart rhythm,NaN,NaN,0.0,4.0,AVblock 3,NaN,NaN,NaN,NaN,listitems,1198,1198.0
2363,6671,Hartritme,heart rhythm,Ritme,320,HD-Hartritme,hemodynamics-heart rhythm,NaN,NaN,0.0,5.0,AVblock 2:1,NaN,NaN,NaN,NaN,listitems,131,131.0
2364,6671,Hartritme,heart rhythm,Ritme,320,HD-Hartritme,hemodynamics-heart rhythm,NaN,NaN,0.0,6.0,AVblock 3:1,NaN,NaN,NaN,NaN,listitems,49,49.0
2365,6671,Hartritme,heart rhythm,Ritme,320,HD-Hartritme,hemodynamics-heart rhythm,NaN,NaN,0.0,7.0,AVBlock 4:1,NaN,NaN,NaN,NaN,listitems,19,19.0
2366,6671,Hartritme,heart rhythm,Ritme,320,HD-Hartritme,hemodynamics-heart rhythm,NaN,NaN,0.0,8.0,Compl. blk,NaN,NaN,NaN,NaN,listitems,182,182.0
2367,6671,Hartritme,heart rhythm,Ritme,320,HD-Hartritme,hemodynamics-heart rhythm,NaN,NaN,0.0,9.0,PAT-block,NaN,NaN,NaN,NaN,listitems,76,76.0
2368,6671,Hartritme,heart rhythm,Ritme,320,HD-Hartritme,hemodynamics-heart rhythm,NaN,NaN,0.0,10.0,ST-depr,NaN,NaN,NaN,NaN,listitems,115,115.0


#Get Rhythm on Admission
##Criteria:

1. First entry made within 1 hour of admission
2. 2 or more entries made within 3 hours of admission
3. All rhythm registered are the same in the first 3 hours (NSR, Sinus Brady, Sinus Tachy and Sinus Arrhythmia are considered the same)
4. Stayed for 4 hours or more

In [13]:
%%bigquery rhythm_on_admission
SELECT admissionid, min_value AS admit_rhythm, num_of_entries, admittedat, first_entry_time FROM
(
    SELECT DISTINCT
      listitems.admissionid
      , MIN(CASE WHEN listitems.valueid=1  -- NSR
              OR listitems.valueid=20 -- Sinus Brady
              OR listitems.valueid=21 -- Sinus Tachy
              OR listitems.valueid=17 -- Sinus Ari
              THEN 1 ELSE listitems.valueid END) AS min_value -- map all sinus rhythm to NSR
      , MAX(CASE WHEN listitems.valueid=1  -- NSR
              OR listitems.valueid=20 -- Sinus Brady
              OR listitems.valueid=21 -- Sinus Tachy
              OR listitems.valueid=17 -- Sinus Ari              
              THEN 1 ELSE listitems.valueid END) AS max_value -- map all sinus rhythm to NSR
      , count(listitems.admissionid) AS num_of_entries
      , admissions.admittedat
      , MIN(listitems.measuredat) AS first_entry_time
    FROM listitems, admissions
    WHERE 
      listitems.itemid = 6671	--Hartritme
      AND listitems.admissionid=admissions.admissionid
      AND admissions.admittedat + 3*60*60*1000 >= listitems.measuredat -- get all entries within 3 hours of admission
      AND admissions.admittedat <= listitems.measuredat -- get all entries within 3 hours of admission
      AND admissions.lengthofstay >=4  -- stayed for 4 hours or more
    GROUP BY listitems.admissionid, admissions.admittedat
)
WHERE min_value=max_value -- same rhythm within first 3 hours
      AND num_of_entries > 1 -- more tan 1 entry of rhythm
      AND admittedat + 1*60*60*1000 >= first_entry_time -- first entry within first hour of admission


In [21]:
rhythm_on_admission.count()

admissionid         12429
admit_rhythm        12429
num_of_entries      12429
admittedat          12429
first_entry_time    12429
dtype: int64

* Sinus 11326
* Afib 754
* Paced 270
* A Flutter 26
* 1 deg AV Block 23



In [22]:
rhythm_on_admission['admit_rhythm'].value_counts()

1     11326
13      754
33      270
14       26
2        23
34        8
15        6
22        4
35        3
12        2
3         2
23        1
5         1
4         1
26        1
25        1
Name: admit_rhythm, dtype: int64

In [16]:
rhythm_on_admission['num_of_entries'].value_counts()

3     8122
2     2987
4     1122
5      150
6       26
7        9
8        8
9        4
13       1
Name: num_of_entries, dtype: int64

##Out of those admitted with SR, find who developed AF

In [18]:
%%bigquery new_af
SELECT listitems.admissionid, MIN(listitems.measuredat) AS first_AF_time, (MIN(listitems.measuredat) - admissions.admittedat)/1000/60/60 AS time_to_AF

FROM listitems, admissions
WHERE listitems.itemid = 6671	--Hartritme
      AND listitems.valueid=13 -- AF
      AND admissions.admissionid=listitems.admissionid
      AND listitems.measuredat > admissions.admittedat
      AND listitems.admissionid IN
                  (SELECT admissionid FROM -- a table with sinus on admission
                      (
                        SELECT admissionid, min_value AS admit_rhythm, num_of_entries, admittedat, first_entry_time FROM
                        (
                            SELECT DISTINCT
                              listitems.admissionid
                              , MIN(CASE WHEN listitems.valueid=1  -- NSR
                                      OR listitems.valueid=20 -- Sinus Brady
                                      OR listitems.valueid=21 -- Sinus Tachy
                                      OR listitems.valueid=17 -- Sinus Ari
                                      THEN 1 ELSE listitems.valueid END) AS min_value -- map all sinus rhythm to NSR
                              , MAX(CASE WHEN listitems.valueid=1  -- NSR
                                      OR listitems.valueid=20 -- Sinus Brady
                                      OR listitems.valueid=21 -- Sinus Tachy
                                      OR listitems.valueid=17 -- Sinus Ari
                                      THEN 1 ELSE listitems.valueid END) AS max_value -- map all sinus rhythm to NSR
                              , count(listitems.admissionid) AS num_of_entries
                              , admissions.admittedat
                              , MIN(listitems.measuredat) AS first_entry_time
                            FROM listitems, admissions
                            WHERE 
                              listitems.itemid = 6671	--Hartritme
                              AND listitems.admissionid=admissions.admissionid
                              AND admissions.admittedat + 3*60*60*1000 >= listitems.measuredat -- get all entries within 3 hours of admission
                              AND admissions.admittedat <= listitems.measuredat -- get all entries within 3 hours of admission
                              AND admissions.lengthofstay >=4  -- stayed for 4 hours or more
                            GROUP BY listitems.admissionid, admissions.admittedat
                        )
                        WHERE min_value=1  -- same SR within first 3 hours
                              AND max_value=1 -- same SR within first 3 hours
                              AND num_of_entries > 1 -- more tan 1 entry of rhythm
                              AND admittedat + 1*60*60*1000 >= first_entry_time -- first entry within first hour of admission
                      )
                  )
GROUP BY listitems.admissionid, admissions.admittedat

#Number of new AF (Sinus rhythm on admission)
1249 out of 11326 patients or 10% have at least 1 label of AF in their ICU stay.

In [19]:
new_af.count()

admissionid      1249
first_AF_time    1249
time_to_AF       1249
dtype: int64

In [20]:
new_af['time_to_AF'].quantile([.05,.25,.5,.75,.95])

0.05      4.823333
0.25     17.250000
0.50     39.516667
0.75     74.466667
0.95    261.360000
Name: time_to_AF, dtype: float64

In [ ]:
new_af.head(10)

,admissionid,first_AF_time,time_to_AF
0,147,98580240000,20.616667
1,240,20024760000,29.150000
2,310,139500000,38.750000
3,748,63060000,17.516667
4,1071,175620000,8.433333
5,1567,124440000,34.566667
6,1619,92998680000,43.316667
7,1812,129000000,35.833333
8,2045,212100000,58.916667
9,2352,303386460000,24.183333


## Out of those, those only have sinus rhythm before the development of first AF

In [26]:
%%bigquery new_af_from_sr
SELECT * FROM
(
    SELECT af.admissionid
            , MAX (CASE WHEN listitems.valueid=1  -- NSR
                    OR listitems.valueid=20 -- Sinus Brady
                    OR listitems.valueid=21 -- Sinus Tachy
                    OR listitems.valueid=17 -- Sinus Ari
                    THEN 1 ELSE listitems.valueid END) AS rhythm_before_AF
            , af.first_AF_time                
            , af.time_to_AF

    FROM listitems,(  -- information on first AF
                    SELECT listitems.admissionid, MIN(listitems.measuredat) AS first_AF_time, (MIN(listitems.measuredat) - admissions.admittedat)/1000/60/60 AS time_to_AF

                    FROM listitems, admissions
                    WHERE listitems.itemid = 6671	--Hartritme
                          AND listitems.valueid=13 -- AF
                          AND admissions.admissionid=listitems.admissionid
                          AND listitems.measuredat > admissions.admittedat
                          AND listitems.admissionid IN
                                      (SELECT admissionid FROM -- a table with sinus on admission
                                          (
                                            SELECT admissionid, min_value AS admit_rhythm, num_of_entries, admittedat, first_entry_time FROM
                                            (
                                                SELECT DISTINCT
                                                  listitems.admissionid
                                                  , MIN(CASE WHEN listitems.valueid=1  -- NSR
                                                          OR listitems.valueid=20 -- Sinus Brady
                                                          OR listitems.valueid=21 -- Sinus Tachy
                                                          OR listitems.valueid=17 -- Sinus Ari
                                                          THEN 1 ELSE listitems.valueid END) AS min_value -- map all sinus rhythm to NSR
                                                  , MAX(CASE WHEN listitems.valueid=1  -- NSR
                                                          OR listitems.valueid=20 -- Sinus Brady
                                                          OR listitems.valueid=21 -- Sinus Tachy
                                                          OR listitems.valueid=17 -- Sinus Ari
                                                          THEN 1 ELSE listitems.valueid END) AS max_value -- map all sinus rhythm to NSR
                                                  , count(listitems.admissionid) AS num_of_entries
                                                  , admissions.admittedat
                                                  , MIN(listitems.measuredat) AS first_entry_time
                                                FROM listitems, admissions
                                                WHERE 
                                                  listitems.itemid = 6671	--Hartritme
                                                  AND listitems.admissionid=admissions.admissionid
                                                  AND admissions.admittedat + 3*60*60*1000 >= listitems.measuredat -- get all entries within 3 hours of admission
                                                  AND admissions.admittedat <= listitems.measuredat -- get all entries within 3 hours of admission
                                                  AND admissions.lengthofstay >=4  -- stayed for 4 hours or more
                                                GROUP BY listitems.admissionid, admissions.admittedat
                                            )
                                            WHERE min_value=1  -- same SR within first 3 hours
                                                  AND max_value=1 -- same SR within first 3 hours
                                                  AND num_of_entries > 1 -- more tan 1 entry of rhythm
                                                  AND admittedat + 1*60*60*1000 >= first_entry_time -- first entry within first hour of admission
                                          )
                                      )
                            GROUP BY listitems.admissionid, admissions.admittedat
                          ) AS af

    WHERE af.admissionid=listitems.admissionid
          AND listitems.itemid = 6671	--Hartritme
          AND listitems.measuredat < af.first_AF_time
    GROUP BY af.admissionid, af.first_AF_time, af.time_to_AF
)
WHERE rhythm_before_AF=1

##New Sinus to AF
###**947 out of 1249 are sinus to AF directly -- CASE**

In [27]:
new_af_from_sr.count()

admissionid         947
rhythm_before_AF    947
first_AF_time       947
time_to_AF          947
dtype: int64

In [28]:
new_af_from_sr['time_to_AF'].quantile([.05,.25,.5,.75,.95])

0.05      4.375000
0.25     13.583333
0.50     33.966667
0.75     62.208333
0.95    217.096667
Name: time_to_AF, dtype: float64

##SR during whole ICU stay

In [23]:
%%bigquery all_SR

SELECT admissionid, min_value AS admit_rhythm, num_of_entries, admittedat, first_entry_time FROM
(
    SELECT DISTINCT
      listitems.admissionid
      , MIN(CASE WHEN listitems.valueid=1  -- NSR
              OR listitems.valueid=20 -- Sinus Brady
              OR listitems.valueid=21 -- Sinus Tachy
              OR listitems.valueid=17 -- Sinus Ari
              THEN 1 ELSE listitems.valueid END) AS min_value -- map all sinus rhythm to NSR
      , MAX(CASE WHEN listitems.valueid=1  -- NSR
              OR listitems.valueid=20 -- Sinus Brady
              OR listitems.valueid=21 -- Sinus Tachy
              OR listitems.valueid=17 -- Sinus Ari
              THEN 1 ELSE listitems.valueid END) AS max_value -- map all sinus rhythm to NSR
      , count(listitems.admissionid) AS num_of_entries
      , admissions.admittedat
      , MIN(listitems.measuredat) AS first_entry_time
    FROM listitems, admissions
    WHERE 
      listitems.itemid = 6671	--Hartritme
      AND listitems.admissionid=admissions.admissionid
      AND admissions.admittedat + admissions.lengthofstay*60*60*1000 >= listitems.measuredat -- get all entries within length of stay
      AND admissions.admittedat <= listitems.measuredat -- get all entries after admission
      AND admissions.lengthofstay >=4  -- stayed for 4 hours or more
    GROUP BY listitems.admissionid, admissions.admittedat
)
WHERE min_value=1  -- same SR in length of stay
      AND max_value=1 -- same SR length of stays
      AND num_of_entries > 1 -- more tan 1 entry of rhythm
      AND admittedat + 1*60*60*1000 >= first_entry_time -- first entry within first hour of admission



##Sinus all along
### **Out of 11326 come in with Sinus, 9485 remained in sinus during the whole ICU stay -- CONTROL**

In [24]:
all_SR.count()

admissionid         9485
admit_rhythm        9485
num_of_entries      9485
admittedat          9485
first_entry_time    9485
dtype: int64

In [ ]:
all_SR.head(10)

,admissionid,admit_rhythm,num_of_entries,admittedat,first_entry_time
0,537,1,7,0,1680000
1,946,1,202,0,1920000
2,1024,1,230,0,240000
3,2025,1,252,0,2880000
4,2273,1,10,0,420000
5,2537,1,526,0,3120000
6,2563,1,79,0,2520000
7,2810,1,19,0,2160000
8,2884,1,18,0,3300000
9,2963,1,87,0,540000
